# Vault - Allow UserPass User to Update Password

## Overview

You might want to allow your users to change their own Userpass passwords. In order to do so, you need to have the appropriate policies in place and assigned to your users. Also, you need to make use of Vault [Template](https://learn.hashicorp.com/tutorials/vault/policy-templating) policies.

## Requirements:

### Set Main Environment Variables

In [2]:
#Set Env. This assumes Dev Mode.
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:8200
MAIN_DIR=$(pwd)
WORK_DIR=config/vault
mkdir -p config/vault/{data,logs,config}

: 1

### Vault Server

* Running Vault Server at http://localhost:8200 - see [Start Vault Server](./100-Setup-Vault.ipynb)

### **Configure username/password**

Enable Userpass Auth Method

In [3]:
vault auth enable userpass

Success! Enabled userpass auth method at: userpass/


: 1

In [5]:
userpass_mount_accessor=$(vault auth list | awk '/^userpass/{print $3}')
echo $userpass_mount_accessor

auth_userpass_8be2b48c


: 1

## Scenario

Assume that the following policy requirements were given:

* Each user can perform all operations on their allocated key/value secret path (`auth/userpass/users/<user_name>`)

### Write and deploy templated ACL policies

In [6]:
vault policy write userpass_update_pw - << EOF
path "auth/userpass/users/{{identity.entity.aliases.${userpass_mount_accessor}.name}}" {
  capabilities = [ "update" ]
  allowed_parameters = {
    "password" = []
  }
}
EOF

Success! Uploaded policy: userpass_update_pw004l


: 1

* `identity.entity.aliases.${userpass_mount_accessor}.name` - Entity alias name for the given mount


### Create user and assign policy

Create user `testuser` and associate with policy `userpass_update_pw` to allow user to only change their own password.

In [7]:
vault write auth/userpass/users/testuser password=test policies=userpass_update_pw

Success! Data written to: auth/userpass/users/testuser


: 1

In [18]:
vault write auth/userpass/users/testuser2 password=test policies=userpass_update_pw

Success! Data written to: auth/userpass/users/testuser2


: 1

## Test the ACL templating policy

Get token for user.

In [8]:
TOKEN_TESTUSER=$(vault login -format=json -method=userpass username=testuser password=test | jq -r .auth.client_token)
echo $TOKEN_TESTUSER

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

s.pryflTfpi7A5WlaNM1VTp547


: 1

Use user token to change `testuser` password from `test` to `newpass`.

In [12]:
VAULT_TOKEN=$TOKEN_TESTUSER vault write auth/userpass/users/testuser password=newpass

Success! Data written to: auth/userpass/users/testuser


: 1

#### API

Login with new password and get token for user.

In [14]:
TOKEN_TESTUSER=$(vault login -format=json -method=userpass username=testuser password=newpass | jq -r .auth.client_token)
echo $TOKEN_TESTUSER

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

s.RDodQQjeG1Nfysd701yc3Dak


: 1

Use user token to change `testuser` password from `test` to `newpass`.

In [15]:
curl -X POST -H "X-Vault-Request: true" -H "X-Vault-Token: ${TOKEN_TESTUSER}" -d '{"password":"newpass123"}' http://localhost:8200/v1/auth/userpass/users/testuser

: 1

Confirm that password has changed.

In [17]:
vault login -format=json -method=userpass username=testuser password=newpass123 | jq -r .auth.client_token

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

s.1Vyc4vbmlP0n5EPAKbO5WwmA


: 1

### Negative Testing

Confirm that you can't change another user's (`testuser2`) password.

In [22]:
curl -X POST -H "X-Vault-Request: true" -H "X-Vault-Token: ${TOKEN_TESTUSER}" -d '{"password":"newpass123"}' http://localhost:8200/v1/auth/userpass/users/testuser2

{"errors":["1 error occurred:\n\t* permission denied\n\n"]}


: 1

## Clean Up

In [ ]:
vault delete /auth/userpass/users/testuser
vault policy delete userpass_update_pw